In [126]:
# Specifying all the imports
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import create_react_agent
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_react_agent, AgentExecutor, ZeroShotAgent
from langchain import LLMChain
from dotenv import load_dotenv
import os

In [127]:
# Loading dotenv and specifying tools
load_dotenv()

@tool
def search_tool(query : str):
    """ 
    Takes a user query and performs an online search to provide
    an answer to the query
    """
    
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def create_embeddings(query : str):
    """
    Uses the content from the file path to create embeddings on the inputted data. Using the query
    provided, perform a similarity search on the query and return the output. If you can not provide a response,
    say I don't know rather than providing an incorrect response.
    """

    file_path = r'C:\GenAI\Final Project\GenAIGroupProject\apple.txt'
    file_path = file_path.strip()
    loader = TextLoader(file_path, encoding= 'utf-8')
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunk = text_splitter.split_documents(documents=document)
    embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
    vectorstore = FAISS.from_documents(chunk, embeddings)

    docs = vectorstore.similarity_search(query)
    return docs[0].page_content

# @tool
# def query_vectorstore(query : str):
#     """
#     Performs a similarity search on the vector store based on the query provided
#     """

#     x = 

    

tools = [Tool(name="search_tool", func=search_tool, description="Tool for performing online search operations"),
         Tool(name="create_embeddings", func=create_embeddings, description="Uses the content from the file path to create embeddings on the inputted data. Using the query provided, perform a similarity search on the query and return the output. If you can not provide a response,say I don't know rather than providing an incorrect response.")]

In [128]:
template = """
You need to use the tools to find the appropriate action to take. Use the embeddings first
to see if you can get relevant responses from the similarity search from the user's query. If you do get relevant
responses, please return the responses from the similarity search. If you can't get relevent results from the embeddings,
use the seatch tool and get the reponse from there. If you still can't find an answer from an online search, please do not make up an
answer and return 'Sorry I can not answer your query' instead.

You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

# Define the query and create the prompt
query = 'sadfidsagigsadifgoidsagfoisagdfidsa'

prompt = PromptTemplate(input_variables=['input','tools', 'agent_scratchpad', 'tool_names'], template=template)

# Initialize the language model
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)


# Create the agent using the language model and the toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Execute the agent with the given input
response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])



> Entering new AgentExecutor chain...
I should start by using the embeddings tool to see if I can find any relevant responses to the gibberish query.

Action: create_embeddings

Action Input: "sadfidsagigsadifgoidsagfoisagdfidsa"
r20210926202209240000320193aaplWearablesHomeandAccessoriesMember20200927202109250000320193aaplWearablesHomeandAccessoriesMember201909292020092600003201932022092520220924xbrlipure0000320193202310012022092400003201932024092920220924000032019320250928202209240000320193usgaapCashMember202209240000320193usgaapMoneyMarketFundsMemberusgaapFairValueInputsLevel1Member202209240000320193usgaapMutualFundMemberusgaapFairValueInputsLevel1Member202209240000320193usgaapFairValueInputsLevel1Member202209240000320193usgaapUSTreasurySecuritiesMemberusgaapFairValueInputsLevel2Member202209240000320193usgaapFairValueInputsLevel2MemberusgaapUSGovernmentAgenciesDebtSecuritiesMember202209240000320193usgaapForeignGovernmentDebtSecuritiesMemberusgaapFairValueInputsLevel2Member202209240

RatelimitException: https://links.duckduckgo.com/d.js?q=sadfidsagigsadifgoidsagfoisagdfidsa%22%0A&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-11395072992083129706432258388145667559&bing_market=wt-wt&ex=-1 202 Ratelimit